# LLM Client Usage Demo

This notebook demonstrates how to use the `llm_client.py` module to create and interact with various LLM providers (OpenAI, Deepseek, Gemini) via the `LLMFactory`.

## 2. Import LLM Client Classes

In [1]:
import os
import json
from dotenv import load_dotenv

# ensure repo root is on path
import sys
sys.path.append(os.path.abspath(os.path.join("..", "")))

from src.core.llm_client import ModelProvider, LLMFactory, OpenAIClient, DeepseekClient, GeminiClient

## 3. Configure Environment Variables

In [2]:
# either set in this cell (not recommended for secrets) or create a .env file
load_dotenv()

# example assignments (uncomment and fill with real keys if desired):
# os.environ['OPENAI_API_KEY'] = 'your-openai-key'
# os.environ['DEEPSEEK_API_KEY'] = 'your-deepseek-key'
# os.environ['GEMINI_API_KEY'] = 'your-gemini-key'

print("keys loaded?", 
      'OPENAI_API_KEY' in os.environ, 
      'DEEPSEEK_API_KEY' in os.environ, 
      'GEMINI_API_KEY' in os.environ)

keys loaded? True True True


## 4. Instantiate Clients with Different Providers

In [5]:
# create clients via the factory
openai_client = LLMFactory.create_client(ModelProvider.OPENAI, temperature=0.3)
deepseek_client = LLMFactory.create_client(ModelProvider.DEEPSEEK, model_name='deepseek-chat', temperature=0.5)
gemini_client = LLMFactory.create_client(ModelProvider.GOOGLE, model_name='gemini-flash-latest', temperature=0.7)

print(openai_client, deepseek_client, gemini_client)

<src.core.llm_client.OpenAIClient object at 0x000001DABC03FF20> <src.core.llm_client.DeepseekClient object at 0x000001DAB8721940> <src.core.llm_client.GeminiClient object at 0x000001DAB972A540>


## 4.1 Notes on warnings

## 5. Generate Text with Each Client

You may see output like object memory addresses or deprecation warnings. The memory addresses (e.g. `<src.core.llm_client.OpenAIClient object at ...>`) are just Python's default `repr` for the instances; the above change prints a friendly summary instead.

Tqdm warnings can be resolved by installing and enabling `ipywidgets` in your Jupyter environment. The Google package deprecation message indicates you should migrate from `google.generativeai` to `google.genai` when updating your dependencies.

In [9]:
prompt = "Please give me a two‑sentence inspirational quote about learning with AI.  Make it at least 50 words."
system = "You are a helpful assistant."

print("--- OpenAI ---")
print(openai_client.generate_text(prompt, system_prompt=system))

print("\n--- Deepseek ---")
print(deepseek_client.generate_text(prompt, system_prompt=system))

print("\n--- Gemini ---")
print(gemini_client.generate_text(prompt, system_prompt=system))

--- OpenAI ---
Embrace the journey of learning with AI, where each interaction is a step towards unlocking the vast potential of human creativity and innovation. As we blend our curiosity with the boundless capabilities of artificial intelligence, we forge a path to a future where knowledge knows no bounds, and every challenge becomes an opportunity for growth and discovery.

--- Deepseek ---
Embrace the boundless curiosity of artificial intelligence as your companion, for it transforms the vast, uncharted ocean of human knowledge into a navigable river, guiding you toward shores of understanding you never dreamed you could reach. Let this synergy of silicon and spirit remind you that the greatest learning occurs not in replacing our wonder, but in using these brilliant tools to amplify it, propelling both our questions and our capabilities toward a brighter, more enlightened horizon.

--- Gemini ---
Embracing the power of artificial intelligence allows us to unlock hidden potential wi

## 6. Generate JSON with Each Client

In [ ]:
# define a simple schema
schema = {
    "type": "object",
    "properties": {
        "message": {"type": "string"},
        "length": {"type": "integer"}
    },
    "required": ["message", "length"]
}

json_prompt = "Provide a JSON object containing an encouraging message and its character count."

for client, name in [(openai_client, 'OpenAI'), (deepseek_client, 'Deepseek'), (gemini_client, 'Gemini')]:
    print(f"--- {name} JSON response ---")
    try:
        result = client.generate_json(json_prompt, schema=schema, max_tokens=80)
        print(result)
    except Exception as e:
        print("error", e)

## 7. Handle Errors and Validations

In [ ]:
# missing key example
os.environ.pop('OPENAI_API_KEY', None)
try:
    LLMFactory.create_client(ModelProvider.OPENAI)
except ValueError as e:
    print("Expected error for missing key:", e)

# JSON parsing failure simulation: send bad prompt
os.environ['OPENAI_API_KEY'] = 'dummy'
class DummyClient(OpenAIClient):
    def generate_text(self, prompt, max_tokens=None, system_prompt=None):
        return "not a json"

bad_client = DummyClient(api_key='abc', model_name='gpt-4o')
try:
    bad_client.generate_json('anything', schema={})
except ValueError as e:
    print("Caught JSON parse error:", e)